# 1. Background of Excellence in Research Award (Phase II)

![Example Image](https://i.ibb.co/2WqHdXy/Phase-II-partner-logo-bar-v2.png)

Climate change is a globally relevant, urgent, and multi-faceted issue heavily impacting many industries and aspects of public life. Participants in Phase II will have the opportunity to examine the climate change from different perspectives. Participants will choose to explore one dataset among several, spanning sectors including healthcare, energy and environmental protection. Participants will also have opportunities to take deeper dives into their dataset and tackle a range of impactful real-world tasks.

Phase II participants will be able to choose one of three research tracks to explore:
- ->US Environmental Protection Agency (EPA): weather, air pollutant, and census data
- MIT Critical Data: CDC county level COVID data
- Climate Change AI: Fine grained building energy usage data

Description of the data:

This dataset represents daily air quality measurements in the United States for 2019 and 2020 in EPA’s Air Quality System (AQS, https://www.epa.gov/aqs) database in which both PM2.5 and ozone are measured concurrently.  These PM2.5 and ozone concentration data are joined with locational, meteorological, demographic information, and concentrations of other major air quality pollutants when available.  All of the data were downloaded from AQS with the exception of four demographic parameters (people of color, low income, linguistically isolated, and less than high school education) which come from EPA’s EJSCREEN tool (https://www.epa.gov/ejscreen).  These demographic parameters are at the Census "block group" level (area defined by the Census Bureau that usually has between 600 and 3,000 people) and listed in fractional units for the block group containing the monitor location. 

# 2. Import Libraries, Define Functions

In [2]:
import pandas as pd
import os

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

# set the default precision to one decimal place
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
def load_data():
    file_paths = []
    for dirname, _, filenames in os.walk('epa'):
        for filename in filenames:
            file_paths.append(os.path.join(dirname, filename))
    
    # Load Data
    air_2019 = pd.read_excel(file_paths[-1])
    air_2020 = pd.read_excel(file_paths[-2])
    
    # Specify STATE and COUNTY (Los Angeles and Riverside)
    air_LA_2019 = air_2019[(air_2019['STATE'] == 'California') & (air_2019['COUNTY'] == 'Los Angeles')]
    air_LA_2020 = air_2020[(air_2020['STATE'] == 'California') & (air_2020['COUNTY'] == 'Los Angeles')]
    
    air_RS_2019 = air_2019[(air_2019['STATE'] == 'California') & (air_2019['COUNTY'] == 'Riverside')]
    air_RS_2020 = air_2020[(air_2020['STATE'] == 'California') & (air_2020['COUNTY'] == 'Riverside')]
    
    # Merge 2019 and 2020 datasets
    df_LA_merged = pd.concat([air_LA_2019, air_LA_2020])
    df_RS_merged = pd.concat([air_RS_2019, air_RS_2020])

    return df_LA_merged, df_RS_merged

def preprocess(df, df_RS):
    # Change DATE column to pandas datetime
    df['datetime'] = pd.to_datetime(df['DATE'])
    # Sort data by datetime
    df_sorted = df.sort_values('datetime')
    # To tackle duplicates, group by datetime
    df_grouped = df_sorted.groupby('datetime').mean()
    
    # Drop unnecessary columns
    # Since we are only looking at a specific location, geographic data will be constant
    # Demographic data will also be constant
    # Other atmospheric data are too sparse with too many empty records, so we will drop them
    df_dropped = df_grouped.drop(['LATITUDE', 'LONGITUDE','PEOPLE_OF_COLOR_FRACTION',
                                  'LOW_INCOME_FRACTION', 'LINGUISTICALLY_ISOLATED_FRACTION',
                                  'LESS_THAN_HS_ED_FRACTION', 'LEAD_UG_PER_CUBIC_METER', 'BENZENE_PPBC',
                                  'WIND_DIRECTION', 'RELATIVE_HUMIDITY'], axis=1)
    
    # Preprocess/fill in data for SO2
    df_dropped = preprocess_so2(df_dropped)
    # Preprocess/fill in data for Temperature
    df_dropped = preprocess_temp(df_dropped, df_RS)
    
    # Select the columns with null values
    null_counts = df_dropped.isnull().sum()
    null_columns = null_counts[null_counts > 0].index
    # Impute missing values using SimpleImputer from sklearn.impute module. Here, we will use mean imputation for simplicity
    imputer = SimpleImputer(strategy='mean')
    # Apply mean imputation to the selected columns with null values
    df_dropped[null_columns] = imputer.fit_transform(df_dropped[null_columns])
    
    # Resample the DataFrame to include all dates and forward-fill missing values
    df_dropped = df_dropped.resample('D').ffill()
    # Fill in any remaining missing values with the previous value
    df_dropped = df_dropped.fillna(method='ffill')
    
    # Re-organize columns so that target variable is first column
    df_preprocessed = df_dropped.reindex(columns=['PM25_UG_PER_CUBIC_METER','TEMPERATURE_CELSIUS',
                                             'WIND_SPEED_METERS_PER_SECOND',
                                             'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'])
    
    return df_preprocessed

def preprocess_so2(df):
    # Extract the 'NO2' and 'SO2' columns
    no2_values = df['NO2_PPB']
    so2_values = df['SO2_PPB']

    # Standardize the 'NO2' and 'SO2' values
    scaler = StandardScaler()
    no2_values_norm = scaler.fit_transform(no2_values.values.reshape(-1, 1))
    so2_values_norm = scaler.fit_transform(so2_values.values.reshape(-1, 1))

    # Replace missing 'SO2' values with the corresponding 'NO2' values
    so2_values_norm = pd.Series(so2_values_norm.flatten(), index=so2_values.index)
    no2_values_norm = pd.Series(no2_values_norm.flatten(), index=no2_values.index)
    so2_values_norm = so2_values_norm.fillna(no2_values_norm)

    # Inverse transform the normalized 'SO2' values
    so2_values = scaler.inverse_transform(so2_values_norm.values.reshape(-1, 1))

    # Update the 'SO2' column in the DataFrame
    df['SO2_PPB'] = so2_values
    
    return df

def preprocess_temp(df, df_RS):
    df_RS['datetime'] = pd.to_datetime(df_RS['DATE'])
    df_RS_sorted = df_RS.sort_values('datetime')
    air_RS = df_RS_sorted.groupby('datetime').mean()

    for i, _ in df[df['TEMPERATURE_CELSIUS'].isna()].iterrows():
        df.loc[i]['TEMPERATURE_CELSIUS'] = air_RS.loc[i]['TEMPERATURE_CELSIUS']
    
    return df

# 3. Load and Preprocess Data

In [4]:
df_LA, df_RS = load_data()

In [5]:
df_preprocessed = preprocess(df_LA, df_RS)

In [6]:
df_preprocessed

,PM25_UG_PER_CUBIC_METER,TEMPERATURE_CELSIUS,WIND_SPEED_METERS_PER_SECOND,OZONE_PPM,NO2_PPB,CO_PPM,SO2_PPB
datetime,,,,,,,
2019-01-01,9.55,6.60,1.43,0.03,13.36,0.45,0.45
2019-01-02,9.55,6.60,1.43,0.03,13.36,0.45,0.45
2019-01-03,8.32,10.55,1.42,0.03,25.79,0.62,0.44
2019-01-04,12.60,8.61,2.04,0.02,27.74,0.76,0.61
2019-01-05,13.85,8.25,2.35,0.03,20.11,0.65,0.43
...,...,...,...,...,...,...,...
2020-12-26,18.26,12.95,0.85,0.03,18.31,0.65,0.18
2020-12-27,19.60,14.05,1.53,0.03,12.86,0.38,0.04
2020-12-28,2.80,13.15,1.53,0.03,9.59,0.21,0.00


In [12]:
# Output preprocessed dataframe, since loading raw every time is both time consuming as well as redundant
df_preprocessed.to_csv("data/df_preprocessed.csv")